### Semblance calculation
Example of semblance calculation

In [1]:
import sys

import numpy as np
import matplotlib.pyplot as plt

sys.path.append('..')

from seismicpro.batchflow import ImagesBatch, Dataset, V, B, Pipeline, D
from seismicpro import FieldIndex, CustomIndex, SeismicDataset, seismic_plot, SeismicBatch

This daaset contains one field with 1980 traces.

In [2]:
path_raw = '/data/H2_PAL/SEMB/H2_PAL_va_supergather_11.sgy'

index_raw = FieldIndex(name='raw', extra_headers=['offset', 'CDP'], path=path_raw)
ix_raw = CustomIndex(index_raw, index_name='CDP')

In [3]:
dset = SeismicDataset(ix_raw, batch_class=SeismicBatch)

In [6]:
pipeline = (
    dset.p
    .load(fmt='segy', components='raw', tslice=slice(3000))
    .sort_traces(src='raw', dst='raw', sort_by='offset')
    .calculate_semblance(src='raw', dst='semblance_hard', velocity=[1200, 6000], 
                         step=30, window=51)
)
pipeline.run(1, n_epochs=5, profile=True, bar='n');

In [7]:
pipeline.elapsed_time

6.583434820175171

In [8]:
time_df = pipeline.show_profile_info(per_iter=True, detailed=False)
time_df.loc[(slice(1, 5), 'calculate_semblance #2'), :]

,,total_time,pipeline_time,batch_id
iter,action,,,
1,calculate_semblance #2,3.678452,3.673242,1.400623e+14
2,calculate_semblance #2,0.463226,0.459226,1.400623e+14
3,calculate_semblance #2,0.477041,0.468926,1.400623e+14
4,calculate_semblance #2,0.483625,0.479463,1.400620e+14
5,calculate_semblance #2,0.465975,0.461832,1.400620e+14
